In [4]:
import cv2
import numpy as np

class BuoyNavigator:
    def __init__(self):
        # Define HSV color ranges for red and green
        self.red_lower1 = np.array([0, 100, 100])
        self.red_upper1 = np.array([10, 255, 255])
        self.red_lower2 = np.array([160, 100, 100])
        self.red_upper2 = np.array([180, 255, 255])
        self.green_lower = np.array([40, 50, 50])
        self.green_upper = np.array([90, 255, 255])

    def process_frame(self, frame):
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Red mask (two ranges because red wraps around HSV space)
        red_mask1 = cv2.inRange(hsv, self.red_lower1, self.red_upper1)
        red_mask2 = cv2.inRange(hsv, self.red_lower2, self.red_upper2)
        red_mask = cv2.bitwise_or(red_mask1, red_mask2)

        # Green mask
        green_mask = cv2.inRange(hsv, self.green_lower, self.green_upper)

        # Find contours for red and green
        red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        green_contours, _ = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        red_boxes = [cv2.boundingRect(cnt) for cnt in red_contours if cv2.contourArea(cnt) > 300]
        green_boxes = [cv2.boundingRect(cnt) for cnt in green_contours if cv2.contourArea(cnt) > 300]

        height, width = frame.shape[:2]
        direction = "Stop!"

        if red_boxes and green_boxes:
            red_x = min([x for (x, y, w, h) in red_boxes])
            green_x = max([x for (x, y, w, h) in green_boxes])
            if red_x < width // 2 and green_x > width // 2:
                direction = "Keep the course!"
            else:
                direction = "Stop!"
        elif green_boxes:
            direction = "Turn Left!"
        elif red_boxes:
            direction = "Turn Right!"

        # Draw boxes and direction
        for (x, y, w, h) in red_boxes:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, "Red Buoy", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        for (x, y, w, h) in green_boxes:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, "Green Buoy", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        
        #text
        # Get text size
        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 1.8  # make it bigger
        thickness = 4
        text_size = cv2.getTextSize(direction, font, scale, thickness)[0]

        # Calculate position to center it
        x = (frame.shape[1] - text_size[0]) // 2
        y = 70  # vertical position from top

        # Draw the text
        cv2.putText(frame, direction, (x, y), font, scale, (0, 0, 0), thickness, cv2.LINE_AA)
        return frame



    def run_on_video(self, video_path, output_path):
        cap = cv2.VideoCapture(video_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            processed = self.process_frame(frame)
            out.write(processed)

        cap.release()
        out.release()
        return output_path

# Run the system on uploaded video
navigator = BuoyNavigator()
input_video = "/content/empty sea.mp4"
output_video = "/content/Detection_video_Stop.mp4"
navigator.run_on_video(input_video, output_video)


ModuleNotFoundError: No module named 'cv2'

In [1]:
cap = cv2.VideoCapture(output_video)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("Processed Video", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined